**EXTRACTING FEATURES OF VIDEO**


**Video Captioning** is a two-part project. In the first part, the features of the video are extracted.


**Video** is One can say a video is a list of images right? So for a video in the data set each and every image called frame is extracted from the video

In [1]:
!pip install kaggle

In [2]:
!kaggle datasets download -d sarthakjain004/msvd-clips

Dataset URL: https://www.kaggle.com/datasets/sarthakjain004/msvd-clips
License(s): unknown
 99% 1.70G/1.71G [00:24<00:00, 155MB/s]
100% 1.71G/1.71G [00:25<00:00, 73.5MB/s]


In [3]:
from zipfile import ZipFile
dataset = '/content/msvd-clips.zip'
with ZipFile(dataset, 'r') as Zip:
  Zip.extractall()
  print("The dataset is extracted")

The dataset is extracted


معالجة الفيديو: قبل تغذية الفيديو في النموذج، يجب معالجته أولاً. هذا يعني استخراج الإطارات من الفيديو وتحويلها إلى مجموعة من الميزات. هذا يمكن أن يتم باستخدام نموذج مثل VGG16 أو InceptionV3 لاستخراج الميزات من كل إطار.
تحضير النص المستهدف: يجب تحويل النص المستهدف (الجملة الوصفية) إلى تسلسل من الرموز. هذا يمكن أن يتم باستخدام Tokenizer في Keras، كما يمكن تحويل الرموز إلى تمثيلات واحدة الساخنة (one-hot) لاستخدامها كمدخلات للفكاك.
تدريب النموذج: بمجرد أن يكون لديك البيانات المعالجة، يمكنك تدريب النموذج باستخدام البيانات التدريبية. يمكنك أيضا استخدام التوقف المبكر (EarlyStopping) لمنع التدريب من الاستمرار لوقت طويل جداً إذا لم يتحسن الأداء.
التنبؤ: بعد تدريب النموذج، يمكنك استخدامه لتوليد جمل وصفية للفيديوات الجديدة. هذا يتطلب بعض الخطوات الإضافية لتحويل الإخراج من النموذج (توزيع الاحتمالات على الرموز) إلى جملة نصية فعلية.
تقييم الأداء: أخيراً، يجب تقييم أداء النموذج باستخدام بعض المقاييس، مثل BLEU أو METEOR، لقياس مدى جودة الجمل الوصفية التي يولدها النموذج

In [4]:
import shutil
import numpy as np
import cv2
import os
from keras.applications.vgg16 import VGG16
from keras.models import Model

base_path = '/content/YouTubeClips'

def extract_feats_pretrained_cnn():
    def video_to_frames(video):
        path = os.path.join(base_path, 'temporary_images')
        if os.path.exists(path):
            shutil.rmtree(path)
        os.makedirs(path)
        video_path = os.path.join(base_path, video)
        count = 0
        image_list = []
        cap = cv2.VideoCapture(video_path)
        while cap.isOpened():
            ret, frame = cap.read()
            if ret is False:
                break
            cv2.imwrite(os.path.join(base_path, 'temporary_images', 'frame%d.jpg' % count), frame)
            image_list.append(os.path.join(base_path, 'temporary_images', 'frame%d.jpg' % count))
            count += 1

        cap.release()
        cv2.destroyAllWindows()
        return image_list

    def model_cnn_load():
        model = VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
        out = model.layers[-2].output
        model_final = Model(inputs=model.input, outputs=out)
        return model_final

    def load_image(path):
        img = cv2.imread(path)
        img = cv2.resize(img, (224, 224))
        return img

    model = model_cnn_load()
    print('Model loaded')

    if not os.path.isdir(os.path.join(base_path, 'feat')):
        os.mkdir(os.path.join(base_path, 'feat'))

    video_list = os.listdir(base_path)
    video_list = [video for video in video_list if video.endswith('.avi')]

    for video in video_list:
        video_id = video.split(".")[0]
        print(video_id)
        print(f'Processing video {video}')

        image_list = video_to_frames(video)
        samples = np.round(np.linspace(
            0, len(image_list) - 1, 80))
        image_list = [image_list[int(sample)] for sample in samples]
        images = np.zeros((len(image_list), 224, 224, 3))
        for i in range(len(image_list)):
            img = load_image(image_list[i])
            images[i] = img
        images = np.array(images)
        fc_feats = model.predict(images, batch_size=128)
        img_feats = np.array(fc_feats)
        shutil.rmtree(os.path.join(base_path, 'temporary_images'))

        outfile = os.path.join(base_path, 'feat', video + '.npy')
        np.save(outfile, img_feats)

if __name__ == "__main__":
    extract_feats_pretrained_cnn()


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 66ms/step
eTnlw7v8ea0_36_50
Processing video eTnlw7v8ea0_36_50.avi
1/1 [==============================] - 0s 66ms/step
NjCqtzZ3OtU_303_307
Processing video NjCqtzZ3OtU_303_307.avi
1/1 [==============================] - 0s 67ms/step
m1NR0uNNs5Y_224_230
Processing video m1NR0uNNs5Y_224_230.avi
1/1 [==============================] - 0s 94ms/step
16iNk1hLJt4_37_40
Processing video 16iNk1hLJt4_37_40.avi
1/1 [==============================] - 0s 97ms/step
omIPdpxg--4_39_46
Processing video omIPdpxg--4_39_46.avi
1/1 [==============================] - 0s 67ms/step
L6dEUQ6WiHY_136_142
Processing video L6dEUQ6WiHY_136_142.avi
1/1 [==============================] - 0s 67ms/step
hJFBXHtxKIc_310_315
Processing video hJFBXHtxKIc_310_315.avi
1/1 [==============================] - 0s 66ms/step
PeUHy0A1GF0_62_66
Processing video PeUHy0A1GF0_62_66.avi
1/1 [==============================] - 0s 66ms/step
5OuYhq6Zl

In [ ]:
"""import shutil
import numpy as np
import cv2
import os
from keras.applications.vgg16 import VGG16
from keras.models import Model

base_path = '/content/YouTubeClips'

def video_to_frames(video):
    path = os.path.join(base_path, 'temporary_images')
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)
    video_path = os.path.join(base_path, video)
    count = 0
    image_list = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if ret is False:
            break
        cv2.imwrite(os.path.join(base_path, 'temporary_images', 'frame%d.jpg' % count), frame)
        image_list.append(os.path.join(base_path, 'temporary_images', 'frame%d.jpg' % count))
        count += 1

    cap.release()
    cv2.destroyAllWindows()
    return image_list

def model_cnn_load():
    model = VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
    out = model.layers[-2].output
    model_final = Model(inputs=model.input, outputs=out)
    return model_final

def load_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))
    return img

def extract_features(video, model):
    video_id = video.split(".")[0]
    print(video_id)
    print(f'Processing video {video}')

    image_list = video_to_frames(video)
    samples = np.round(np.linspace(
        0, len(image_list) - 1, 80))
    image_list = [image_list[int(sample)] for sample in samples]
    images = np.zeros((len(image_list), 224, 224, 3))
    for i in range(len(image_list)):
        img = load_image(image_list[i])
        images[i] = img
    images = np.array(images)
    fc_feats = model.predict(images, batch_size=128)
    img_feats = np.array(fc_feats)
    shutil.rmtree(os.path.join(base_path, 'temporary_images'))
    return img_feats

def extract_feats_pretrained_cnn():
    model = model_cnn_load()
    print('Model loaded')

    if not os.path.isdir(os.path.join(base_path, 'feat')):
        os.mkdir(os.path.join(base_path, 'feat'))

    video_list = os.listdir(base_path)
    video_list = [video for video in video_list if video.endswith('.avi')]

    for video in video_list:
        outfile = os.path.join(base_path, 'feat', video + '.npy')
        img_feats = extract_features(video, model)
        np.save(outfile, img_feats)

if __name__ == "__main__":
    extract_feats_pretrained_cnn()
"""

553467096/553467096 [==============================] - 5s 0us/step
Model loaded
6Lzq1HN33lE_5_15
Processing video 6Lzq1HN33lE_5_15.avi
1/1 [==============================] - 50s 50s/step
6t0BpjwYKco_254_258
Processing video 6t0BpjwYKco_254_258.avi
1/1 [==============================] - 50s 50s/step
zTn-nNj5Bng_8_19
Processing video zTn-nNj5Bng_8_19.avi
1/1 [==============================] - 52s 52s/step
TIubRrkku-4_1_10
Processing video TIubRrkku-4_1_10.avi
1/1 [==============================] - 50s 50s/step
Jxw16bqzVqE_18_28
Processing video Jxw16bqzVqE_18_28.avi
1/1 [==============================] - 51s 51s/step
MrMG15y3rWU_5_13
Processing video MrMG15y3rWU_5_13.avi
1/1 [==============================] - 49s 49s/step
ACK7SekJWts_140_150
Processing video ACK7SekJWts_140_150.avi
1/1 [==============================] - 51s 51s/step
klFyrnrUSck_63_73
Processing video klFyrnrUSck_63_73.avi
1/1 [==============================] - 51s 51s/step
BnJUWwSx1kE_11_22
Processing video BnJUWwSx1kE_1

الدالة shutil.rmtree() هي دالة في مكتبة shutil في Python تُستخدم لحذف مجلد وكل محتوياته.

لدالة os.path.join(base_path, 'temporary_images') تقوم بدمج base_path و ‘temporary_images’ لإنشاء مسار كامل إلى المجلد ‘temporary_images’.

بشكل عام، يتم استخدام هذا الكود للتأكد من أن المجلد ‘temporary_images’ فارغ قبل بدء معالجة الفيديو الجديد. إذا كان المجلد موجودًا بالفعل ويحتوي على بعض الصور، فسيتم حذفه وإعادة إنشاؤه لضمان عدم وجود صور قديمة تتداخل مع الفيديو الجديد

يمكنك استخدام أي نموذج مُعَدّ مُسبقًا آخر بدلاً من VGG16. توجد العديد من النماذج المُعَدّة مُسبقًا في Keras مثل ResNet, InceptionV3, Xception, و EfficientNet. كل هذه النماذج يمكن استخدامها لاستخراج الميزات من الصور.

لتحميل نموذج مُعَدّ مُسبقًا آخر، يمكنك استبدال الكود في الدالة model_cnn_load بالكود المناسب للنموذج الذي تريد استخدامه. على سبيل المثال،

In [ ]:
'''from tensorflow.keras.applications.resnet50 import ResNet50

def model_cnn_load():
    model = ResNet50(weights="imagenet", include_top=True)
    out = model.layers[-2].output
    model_final = Model(inputs=model.input, outputs=out)
    return model_final'''

يرجى ملاحظة أن كل نموذج يتطلب حجمًا معينًا للصورة كمدخلات. على سبيل المثال، نموذج VGG16 يتطلب صور بحجم 224x224، بينما نموذج InceptionV3 يتطلب صور بحجم 299x299. لذا، قد تحتاج إلى تعديل الدالة load_image لتغيير حجم الصورة وفقًا للنموذج الذي تستخدمه.

تحميل ومعالجة  بيانات التدريب لنموذج التعلم العميق

In [ ]:
#!kaggle datasets download -d vtrnanh/msvd-dataset-corpus

Dataset URL: https://www.kaggle.com/datasets/vtrnanh/msvd-dataset-corpus
License(s): unknown
  0% 0.00/3.09M [00:00<?, ?B/s]
100% 3.09M/3.09M [00:00<00:00, 213MB/s]


In [ ]:
"""from zipfile import ZipFile
dataset = '/content/msvd-dataset-corpus.zip'
with ZipFile(dataset, 'r') as Zip:
  Zip.extractall()
  print("The dataset is extracted")"""

The dataset is extracted


في الكود التالي ، تتم معالجة البيانات وتخزينها في الذاكرة، وليس في ملف. هذا يعني أن البيانات متاحة فقط أثناء تشغيل البرنامج، ولكنها لن تكون متاحة بعد إغلاق البرنامج أو إعادة تشغيله.



لنقم بتوضيح الخطوات التي يقوم بها الكود:



1.   يتم تحميل البيانات من الملف training_label.json، وهذا الملف يحتوي على التعليقات التوضيحية للفيديوهات.

2.   يتم تكرار البيانات واختيار التعليقات التوضيحية التي تحتوي على أكثر من 6 كلمات وأقل من 10 كلمات.


3.   يتم تقسيم البيانات إلى مجموعتين: مجموعة التدريب ومجموعة التحقق.

4.   يتم تحويل الكلمات في التوصيفات التوضيحية إلى أرقام باستخدام الأداة Tokenizer.


5.   يتم تحميل ميزات الفيديو من الملفات .npy الموجودة في المجلد /content/YouTubeClips/feat وتخزينها في الذاكرة في القاموس x_data.








In [7]:
import json
import os
import random

from keras.preprocessing.text import Tokenizer
import numpy as np

# mentioning the train test split
train_split = 0.85
def preprocessing():
  train_path='/content/data/training_data'
  TRAIN_LABEL_PATH = os.path.join(train_path, 'training_label.json')
  # loading the json file for training
  with open(TRAIN_LABEL_PATH) as data_file:
    y_data = json.load(data_file)
    # train_list contains all the captions with their video ID
    # vocab_list contains all the vocabulary from training data
  train_list = []
  vocab_list = []
  for y in y_data:
    for caption in y['caption']:
      caption = "<bos> " + caption + " <eos>"
      # we are only using sentences whose length lie between 6 and 10
      if len(caption.split())>10 or len(caption.split())<6:
        continue
      else:
        train_list.append([caption, y['id']])
        #print(len(train_list))
  random.shuffle(train_list)
  training_list = train_list[:int(len(train_list)*train_split)]
  validation_list = train_list[int(len(train_list)*train_split):]
  for train in training_list:
    vocab_list.append(train[0])
    # Tokenizing the words
  tokenizer = Tokenizer(num_words=1500)
  tokenizer.fit_on_texts(vocab_list)
  x_data = {}
  TRAIN_FEATURE_DIR = os.path.join('/content/data/training_data', '/content/YouTubeClips/feat')
  # Loading all the numpy arrays at once and saving them in a dictionary
  for filename in os.listdir(TRAIN_FEATURE_DIR):
    f = np.load(os.path.join(TRAIN_FEATURE_DIR, filename))
    x_data[filename[:-4]] = f
  return training_list, validation_list, x_data, tokenizer


In [8]:
training_list, validation_list, x_data, tokenizer = preprocessing()

In [9]:
training_list

[['<bos> A man is making a soccer goal. <eos>', 'q-a6NEotUX8_5_10.avi'],
 ['<bos> The woman is singing and playing the guitar. <eos>',
  '1N_Ic2pBM1o_2_23.avi'],
 ['<bos> A man is cutting a potato. <eos>', 'veE0E79dEEc_18_45.avi'],
 ['<bos> A bird is dancing. <eos>', 'cJOZp2ZftCw_1_12.avi'],
 ['<bos> a sword fight between two women <eos>', '9OxQ-2gR1DU_18_38.avi'],
 ['<bos> A man unboxes a rifle. <eos>', 'jfrrO5K_vKM_55_65.avi'],
 ['<bos> A yacht moves across the water. <eos>', 'xgIIcPSh4EU_0_6.avi'],
 ['<bos> A boy is trimming grass with scissors. <eos>',
  'IY1Fn796MoY_7_10.avi'],
 ['<bos> A woman is chopping an onion. <eos>', 'klFyrnrUSck_13_19.avi'],
 ['<bos> The boy climed the rock wall. <eos>', 'o_mWZWcm2r4_10_15.avi'],
 ['<bos> A woman is opening her closet. <eos>', 'JIoiUNOXGaI_98_104.avi'],
 ['<bos> A man is drinking coffee. <eos>', 'LwicaralvS0_90_104.avi'],
 ['<bos> The woman is riding the horse. <eos>', 'L8h2DazQZJY_0_10.avi'],
 ['<bos> A man is speaking and walking. <eos>'

حفظ البيانات بعد معالجتها في ملف

In [ ]:
import pickle

# Save tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save x_data
with open('x_data.pickle', 'wb') as handle:
    pickle.dump(x_data, handle, protocol=pickle.HIGHEST_PROTOCOL)


تحميل البيانات من الملفات

In [ ]:

# Load tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load x_data
with open('x_data.pickle', 'rb') as handle:
    x_data = pickle.load(handle)


**MODEL FOR TRAINING**

الكود التالي  يقوم بتدريب نموذج التعلم العميق لتوليد الوصف النصي للفيديو. النموذج هو نموذج الترميز والتشفير (Encoder-Decoder) الذي يستخدم LSTM. الكود يقوم بتحميل البيانات بدفعات، تدريب النموذج، وحفظ النموذج والمعالج المستخدم للنص

هذا الكود يقوم بإنشاء نموذج للتعلم العميق باستخدام الشبكات العصبية المتكررة (RNN) من نوع LSTM للتعامل مع مهمة تتطلب ترجمة الفيديو إلى النص. هذا النموذج يتألف من جزئين رئيسيين: المشفر (Encoder) والفكاك (Decoder).

المشفر (Encoder):

يتلقى المشفر مجموعة من الإطارات (frames) من الفيديو كمدخلات، حيث يتم استخدام time_steps_encoder إطار لكل فيديو، وكل إطار يحتوي على num_encoder_tokens ميزة.
يتم تمرير هذه الإطارات عبر طبقة LSTM، حيث latent_dim هو عدد الخصائص المخفية لـ LSTM.
يتم تجاهل الإخراج الأساسي لـ LSTM ولكن يتم الاحتفاظ بحالاته الداخلية (state_h, state_c) لاستخدامها كحالة أولية للفكاك.
الفكاك (Decoder):

يتلقى الفكاك الجملة المستهدفة كمدخلات، حيث time_steps_decoder هو الطول الأقصى لكل جملة، وكل كلمة في الجملة تمثل بـ num_decoder_tokens رمز.
يتم تمرير هذه الجملة عبر طبقة LSTM أخرى، حيث تكون حالة البداية هي حالات المشفر.
يتم تطبيق طبقة Dense على كل خطوة زمنية من الإخراج، مما ينتج عنه توزيع الاحتمالات عبر num_decoder_tokens الرموز المختلفة.
النموذج النهائي:

يتم تجميع المشفر والفكاك في نموذج واحد، حيث تكون المدخلات هي إطارات الفيديو والجمل المستهدفة، والإخراج هو توزيع الاحتمالات على الرموز المستهدفة.
الكود يستخدم أيضا batch_size الذي يحدد عدد العينات التي ستمرر خلال الشبكة في كل تحديث للوزن.

أخيرا، يتم طباعة ملخص النموذج لعرض بنية النموذج وعدد المعلمات التي يجب تعلمها

الكود يقوم بإنشاء نموذج لشبكة LSTM ثنائية الاتجاه (Bidirectional LSTM). هذا النموذج يتكون من جزئين رئيسيين: المُشفر (Encoder) والمُفك (Decoder).

المُشفر (Encoder): يأخذ المدخلات ويقوم بتحويلها إلى حالة داخلية تحتوي على المعلومات الضرورية للمُفك.
المُفك (Decoder): يأخذ حالة المُشفر ويقوم بتوليد المخرجات.

In [11]:
"""
time_steps_encoder is the number of frames per video we will be using for training
num_encoder_tokens is the number of features from each frame
latent_dim is the number of hidden features for lstm
time_steps_decoder is the maximum length of each sentence
num_decoder_tokens is the final number of tokens in the softmax layer
batch size
"""
import keras
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Input, LSTM, Dense
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import joblib

time_steps_encoder=80
num_encoder_tokens=4096
latent_dim=512
time_steps_decoder=10
num_decoder_tokens=1500
batch_size=320
max_probability = -1

encoder_inputs = Input(shape=(time_steps_encoder, num_encoder_tokens), name="encoder_inputs")
encoder = LSTM(latent_dim, return_state=True,return_sequences=True, name='endcoder_lstm')
_, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]


# Set up the decoder
decoder_inputs = Input(shape=(time_steps_decoder, num_decoder_tokens), name= "decoder_inputs")
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_relu')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, 80, 4096)]           0         []                            
 )                                                                                                
                                                                                                  
 decoder_inputs (InputLayer  [(None, 10, 1500)]           0         []                            
 )                                                                                                
                                                                                                  
 endcoder_lstm (LSTM)        [(None, 80, 512),            9439232   ['encoder_inputs[0][0]']      
                              (None, 512),                                                  

الهدف الأساسي من الكود السابق هو تحميل ومعالجة البيانات التدريبية لنموذج التعلم العميق. النموذج المستخدم هنا هو نموذج الترجمة التسلسلية التلقائية (Seq2Seq)، والذي يتكون من جزئين رئيسيين: المشفر (Encoder) والفكاك (Decoder).

الكود يقوم بالخطوات التالية:

يقوم بتحميل البيانات التدريبية، والتي تتكون من مجموعة من الفيديوات والجمل الوصفية المقابلة لها.
يقوم بتحويل الجمل الوصفية إلى تسلسلات من الرموز باستخدام الـ tokenizer.
يقوم بتوسيع التسلسلات لتكون جميعها بنفس الطول باستخدام الدالة pad_sequences.
يقوم بتحويل التسلسلات إلى تمثيلات واحدة الساخنة (one-hot).
يقوم بتجهيز البيانات للتدريب بحيث يتم تقديم البيانات للنموذج في شكل دفعات.
بشكل عام، الكود يقوم بتحضير البيانات بطريقة تتوافق مع النموذج الذي تم إنشاؤه في الكود السابق، ويتم استخدامه أثناء التدريب لتوفير البيانات للنموذج في شكل دفعات. هذا يساعد في تحسين كفاءة التدريب والأداء العام للنموذج.

هذه الدالة load_datatest تقوم بتحميل ومعالجة البيانات التدريبية للنموذج. هيا نتعرف على كل خطوة بالتفصيل:

تحديد المتغيرات الأولية: يتم تعريف القوائم الفارغة لحفظ بيانات المشفر والفكاك، بالإضافة إلى معرفات الفيديو والتسلسلات.
فصل معرف الفيديو والتسلسلات: يتم فصل معرف الفيديو والتسلسلات من قائمة التدريب وحفظها في قوائم منفصلة.
تحويل التسلسلات إلى رموز وتوسيعها: يتم تحويل التسلسلات النصية إلى رموز باستخدام الـ tokenizer، ثم يتم توسيع التسلسلات لتكون جميعها بنفس الطول باستخدام الدالة pad_sequences.
تحميل البيانات: يتم تحميل البيانات في دورات تكرارية (epochs). في كل دورة، يتم تحميل البيانات من كل فيديو في القائمة. يتم تحويل التسلسلات إلى تمثيلات واحدة الساخنة (one-hot) وتقسيمها إلى بيانات الإدخال والهدف للفكاك. يتم تخزين بيانات الإدخال والهدف في قوائم.
تجهيز البيانات للتدريب: بمجرد أن يصل عدد العينات إلى حجم الدفعة (batch size)، يتم تحويل القوائم إلى مصفوفات numpy وإعادة تعيين القوائم. ثم يتم إرجاع البيانات كإدخال وهدف للنموذج.
بشكل عام، هذه الدالة تقوم بتحميل ومعالجة البيانات بطريقة تتوافق مع النموذج الذي تم إنشاؤه في الكود السابق. يتم استخدامها أثناء التدريب لتوفير البيانات للنموذج في شكل دفعات

In [12]:
def load_datatest(train_path, epochs=100, x_data=x_data, tokenizer=tokenizer, num_decoder_tokens=1500,training_list=training_list, batch_size=32, maxlen=10):
    encoder_input_data = []
    decoder_input_data = []
    decoder_target_data = []
    videoId = []
    videoSeq = []
    # separating the videoId and the video captions
    for idx, cap in enumerate(training_list):
        caption = cap[0]
        videoId.append(cap[1])
        videoSeq.append(caption)
    # converting the captions to tokens and padding them to equal sizes
    train_sequences = tokenizer.texts_to_sequences(videoSeq)
    train_sequences = pad_sequences(train_sequences, padding='post', truncating='post', maxlen=maxlen)
    train_sequences = np.array(train_sequences)
    max_seq_length = train_sequences.shape[1]
    filesize = len(train_sequences)
    X_data = []
    y_data = []
    vCount = 0
    n = 0
    for i in range(epochs):
      for idx in  range(0,filesize):
        n += 1
        encoder_input_data.append(x_data[videoId[idx]])
        y = to_categorical(train_sequences[idx], num_decoder_tokens)
        decoder_input_data.append(y[:-1])
        decoder_target_data.append(y[1:])
        if n == batch_size:
          encoder_input = np.array(encoder_input_data)
          decoder_input = np.array(decoder_input_data)
          decoder_target = np.array(decoder_target_data)
          encoder_input_data = []
          decoder_input_data = []
          decoder_target_data = []
          n = 0
          yield ([encoder_input, decoder_input], decoder_target)

In [13]:
train = load_datatest(training_list)
valid = load_datatest(validation_list)

In [14]:
train_path = "/content/data/training_data"
test_path = "/content/data/testing_data"
batch_size = 320
learning_rate = 0.0007
epochs = 150
latent_dim = 512
num_encoder_tokens = 4096
num_decoder_tokens = 1500
time_steps_encoder = 80
max_probability = -1
save_model_path = 'model_final'
validation_split = 0.15
max_length = 10
search_type = 'greedy'

In [15]:
# Run training
opt = keras.optimizers.Adam(learning_rate=0.0003)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.1, patience=5, verbose=0,mode="auto")
model.compile(metrics=['accuracy'], optimizer=opt, loss='categorical_crossentropy')


In [18]:
validation_steps = len(validation_list)//batch_size
steps_per_epoch = len(training_list)//batch_size
validation_steps


8

In [19]:
steps_per_epoch

45

In [20]:
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='min')

In [21]:
model.fit(train, validation_data=valid, validation_steps=validation_steps,
                  epochs=epochs, steps_per_epoch=steps_per_epoch,
                  callbacks=[reduce_lr, early_stopping])


Epoch 1/150
45/45 [==============================] - 14s 159ms/step - loss: 4.3015 - accuracy: 0.3204 - val_loss: 3.3941 - val_accuracy: 0.3941 - lr: 3.0000e-04
Epoch 2/150
45/45 [==============================] - 5s 115ms/step - loss: 3.4724 - accuracy: 0.3908 - val_loss: 3.3397 - val_accuracy: 0.4023 - lr: 3.0000e-04
Epoch 3/150
45/45 [==============================] - 6s 139ms/step - loss: 3.3444 - accuracy: 0.4094 - val_loss: 3.2053 - val_accuracy: 0.4175 - lr: 3.0000e-04
Epoch 4/150
45/45 [==============================] - 5s 102ms/step - loss: 3.1910 - accuracy: 0.4297 - val_loss: 3.1017 - val_accuracy: 0.4388 - lr: 3.0000e-04
Epoch 5/150
45/45 [==============================] - 5s 117ms/step - loss: 3.0968 - accuracy: 0.4425 - val_loss: 2.9961 - val_accuracy: 0.4379 - lr: 3.0000e-04
Epoch 6/150
45/45 [==============================] - 6s 124ms/step - loss: 3.0451 - accuracy: 0.4481 - val_loss: 3.0027 - val_accuracy: 0.4549 - lr: 3.0000e-04
Epoch 7/150
45/45 [====================

In [22]:
if not os.path.exists(save_model_path):
  os.makedirs(save_model_path)

encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
[decoder_inputs] + decoder_states_inputs,
[decoder_outputs] + decoder_states)


In [23]:
encoder_model.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_inputs (InputLayer  [(None, 80, 4096)]        0         
 )                                                               
                                                                 
 endcoder_lstm (LSTM)        [(None, 80, 512),         9439232   
                              (None, 512),                       
                              (None, 512)]                       
                                                                 
Total params: 9439232 (36.01 MB)
Trainable params: 9439232 (36.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
decoder_model.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 decoder_inputs (InputLayer  [(None, 10, 1500)]           0         []                            
 )                                                                                                
                                                                                                  
 input_2 (InputLayer)        [(None, 512)]                0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, 512)]                0         []                            
                                                                                                  
 decoder_lstm (LSTM)         [(None, 10, 512),            4122624   ['decoder_inputs[0][0]',

In [25]:
# saving the models
encoder_model.save(os.path.join(save_model_path, 'encoder_model.h5'))
decoder_model.save_weights(os.path.join(save_model_path, 'decoder_model_weights.h5'))
with open(os.path.join(save_model_path, 'tokenizer' + str(num_decoder_tokens)), 'wb') as file:
  joblib.dump(tokenizer, file)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
# saving the models
encoder_model.save(os.path.join(save_model_path, 'encoder_model.keras'))
decoder_model.save_weights(os.path.join(save_model_path, 'decoder_model_weights.h5'))
with open(os.path.join(save_model_path, 'tokenizer' + str(num_decoder_tokens)), 'wb') as file:
  joblib.dump(tokenizer, file)


In [27]:
model.save(os.path.join(save_model_path, 'model.h5'))


In [28]:
decoder_model.save(os.path.join(save_model_path, 'decoder_model.keras'))


In [29]:
decoder_model.save(os.path.join(save_model_path, 'decoder_model.h5'))


In [35]:
import shutil
import numpy as np
import cv2
import os
from keras.applications.vgg16 import VGG16
from keras.models import Model

base_path = '/content/data/testing_data/video'

def extract_feats_pretrained_cnn():
    def video_to_frames(video):
        path = os.path.join(base_path, 'temporary_images')
        if os.path.exists(path):
            shutil.rmtree(path)
        os.makedirs(path)
        video_path = os.path.join(base_path, video)
        count = 0
        image_list = []
        cap = cv2.VideoCapture(video_path)
        while cap.isOpened():
            ret, frame = cap.read()
            if ret is False:
                break
            cv2.imwrite(os.path.join(base_path, 'temporary_images', 'frame%d.jpg' % count), frame)
            image_list.append(os.path.join(base_path, 'temporary_images', 'frame%d.jpg' % count))
            count += 1

        cap.release()
        cv2.destroyAllWindows()
        return image_list

    def model_cnn_load():
        model = VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
        out = model.layers[-2].output
        model_final = Model(inputs=model.input, outputs=out)
        return model_final

    def load_image(path):
        img = cv2.imread(path)
        img = cv2.resize(img, (224, 224))
        return img

    model = model_cnn_load()
    print('Model loaded')

    if not os.path.isdir(os.path.join(base_path, 'feat')):
        os.mkdir(os.path.join(base_path, 'feat'))

    video_list = os.listdir(base_path)
    video_list = [video for video in video_list if video.endswith('.avi')]

    for video in video_list:
        video_id = video.split(".")[0]
        print(video_id)
        print(f'Processing video {video}')

        image_list = video_to_frames(video)
        samples = np.round(np.linspace(
            0, len(image_list) - 1, 80))
        image_list = [image_list[int(sample)] for sample in samples]
        images = np.zeros((len(image_list), 224, 224, 3))
        for i in range(len(image_list)):
            img = load_image(image_list[i])
            images[i] = img
        images = np.array(images)
        fc_feats = model.predict(images, batch_size=128)
        img_feats = np.array(fc_feats)
        shutil.rmtree(os.path.join(base_path, 'temporary_images'))

        outfile = os.path.join(base_path, 'feat', video + '.npy')
        np.save(outfile, img_feats)

if __name__ == "__main__":
    extract_feats_pretrained_cnn()


Model loaded
WTf5EgVY5uU_124_128
Processing video WTf5EgVY5uU_124_128.avi
1/1 [==============================] - 0s 196ms/step
klteYv1Uv9A_27_33
Processing video klteYv1Uv9A_27_33.avi
1/1 [==============================] - 0s 67ms/step
UXs3eq68ZjE_250_255
Processing video UXs3eq68ZjE_250_255.avi
1/1 [==============================] - 0s 85ms/step
j2Dhf-xFUxU_13_20
Processing video j2Dhf-xFUxU_13_20.avi
1/1 [==============================] - 0s 90ms/step
3qqEKTPxLNs_1_15
Processing video 3qqEKTPxLNs_1_15.avi
1/1 [==============================] - 0s 66ms/step


هذا الكود يقوم بتحويل الفيديوهات إلى نصوص باستخدام نماذج التعلم العميق. يتم تحميل النماذج والمعجم في البداية، ثم يتم استخدامها لتحويل الفيديوهات إلى نصوص. يتم استخدام الدالة test لاختبار النماذج على مجموعة الاختبار

In [41]:
import functools
import operator
import os
import time

import numpy as np
from keras.layers import Input, LSTM, Dense
from keras.models import Model, load_model
import joblib
search_type = 'beam'

class VideoDescriber:
    def __init__(self):
        # تحميل النماذج والمعجم
        with open(os.path.join(save_model_path, 'tokenizer' + str(num_decoder_tokens)), 'rb') as file:
            self.tokenizer = joblib.load(file)
        self.inf_encoder_model = load_model(os.path.join(save_model_path, 'encoder_model.h5'))
        decoder_inputs = Input(shape=(None, num_decoder_tokens))
        decoder_dense = Dense(num_decoder_tokens, activation='softmax')
        decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
        decoder_state_input_h = Input(shape=(latent_dim,))
        decoder_state_input_c = Input(shape=(latent_dim,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = decoder_dense(decoder_outputs)
        self.inf_decoder_model = Model(
            [decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states)
        self.inf_decoder_model.load_weights(os.path.join(save_model_path, 'decoder_model_weights.h5'))

    def index_to_word(self):
        return {value: key for key, value in self.tokenizer.word_index.items()}

    def greedy_search(self, loaded_array):
        inv_map = self.index_to_word()
        states_value = self.inf_encoder_model.predict(loaded_array.reshape(-1, 80, 4096))
        target_seq = np.zeros((1, 1, 1500))
        sentence = ''
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        for i in range(15):
            output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
            states_value = [h, c]
            output_tokens = output_tokens.reshape(num_decoder_tokens)
            y_hat = np.argmax(output_tokens)
            if y_hat == 0:
                continue
            if inv_map[y_hat] is None:
                break
            else:
                sentence = sentence + inv_map[y_hat] + ' '
                target_seq = np.zeros((1, 1, 1500))
                target_seq[0, 0, y_hat] = 1
        return ' '.join(sentence.split()[:-1])

    def decode_sequence2bs(self, input_seq):
        states_value = self.inf_encoder_model.predict(input_seq)
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        self.beam_search(target_seq, states_value, [], [], 0)
        return decode_seq

    def beam_search(self, target_seq, states_value, prob, path, lens):
        global  max_probability, decode_seq
        node = 2
        output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
        output_tokens = output_tokens.reshape(num_decoder_tokens)
        sampled_token_index = output_tokens.argsort()[-node:][::-1]
        states_value = [h, c]
        for i in range(node):
            if sampled_token_index[i] == 0:
                sampled_char = ''
            else:
                sampled_char = list(self.tokenizer.word_index.keys())[list(self.tokenizer.word_index.values()).index(sampled_token_index[i])]
            MAX_LEN = 12
            if sampled_char != 'eos' and lens <= MAX_LEN:
                p = output_tokens[sampled_token_index[i]]
                if sampled_char == '':
                    p = 1
                prob_new = list(prob)
                prob_new.append(p)
                path_new = list(path)
                path_new.append(sampled_char)

                target_seq = np.zeros((1, 1, num_decoder_tokens))
                target_seq[0, 0, sampled_token_index[i]] = 1.
                self.beam_search(target_seq, states_value, prob_new, path_new, lens + 1)
            else:
                p = output_tokens[sampled_token_index[i]]
                prob_new = list(prob)
                prob_new.append(p)
                p = functools.reduce(operator.mul, prob_new, 1)
                if p > max_probability:
                    decode_seq = path
                    max_probability = p

    def decoded_sentence_tuning(self, decoded_sentence):
        decode_str = []
        filter_string = ['bos', 'eos']
        uni_gram = {}
        last_string = ""
        for idx2, c in enumerate(decoded_sentence):
            if c in uni_gram:
                uni_gram[c] += 1
            else:
                uni_gram[c] = 1
            if last_string == c and idx2 > 0:
                continue
            if c in filter_string:
                continue
            if len(c) > 0:
                decode_str.append(c)
            if idx2 > 0:
                last_string = c
        return decode_str

    def get_test_data(self):
        X_test = []
        X_test_filename = []
        with open(os.path.join(test_path, 'testing_id.txt')) as testing_file:
            lines = testing_file.readlines()
            for filename in lines:
                filename = filename.strip()
                f = np.load(os.path.join('/content/data/testing_data/video', 'feat', filename + '.npy'))
                X_test.append(f)
                X_test_filename.append(filename[:-4])
            X_test = np.array(X_test)
        return X_test, X_test_filename

    def test(self):
        global max_probability
        X_test, X_test_filename = self.get_test_data()
        for search_type in ['greedy', 'beam']:
            with open(os.path.join(test_path, f'test1_{search_type}.txt'), 'w') as file:
                for idx, x in enumerate(X_test):
                    file.write(X_test_filename[idx] + ',')
                    start = time.time()
                    if search_type == 'greedy':
                        decoded_sentence = self.greedy_search(x.reshape(-1, 80, 4096))
                    else:
                        decoded_sentence = self.decode_sequence2bs(x.reshape(-1, 80, 4096))
                        decode_str = self.decoded_sentence_tuning(decoded_sentence)
                        decoded_sentence = ' '.join(decode_str)
                    file.write(decoded_sentence + ',{:.2f}'.format(time.time() - start))
                    file.write('\n')

                    max_probability = -1

if __name__ == "__main__":
    video_to_text = VideoDescriber()
    video_to_text.test()


1/1 [==============================] - 0s 22ms/step


In [42]:
import functools
import operator
import os
import time

import numpy as np
from keras.layers import Input, LSTM, Dense
from keras.models import Model, load_model
import joblib
search_type = 'beam'

class VideoDescriber:
    def __init__(self):
        with open(os.path.join(save_model_path, 'tokenizer' + str(num_decoder_tokens)), 'rb') as file:
            self.tokenizer = joblib.load(file)
        self.inf_encoder_model = load_model(os.path.join(save_model_path, 'encoder_model.h5'))
        decoder_inputs = Input(shape=(None, num_decoder_tokens))
        decoder_dense = Dense(num_decoder_tokens, activation='softmax')
        decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
        decoder_state_input_h = Input(shape=(latent_dim,))
        decoder_state_input_c = Input(shape=(latent_dim,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = decoder_dense(decoder_outputs)
        self.inf_decoder_model = Model(
            [decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states)
        self.inf_decoder_model.load_weights(os.path.join(save_model_path, 'decoder_model_weights.h5'))

    def index_to_word(self):
        return {value: key for key, value in self.tokenizer.word_index.items()}

    def greedy_search(self, loaded_array):
        inv_map = self.index_to_word()
        states_value = self.inf_encoder_model.predict(loaded_array.reshape(-1, 80, 4096))
        target_seq = np.zeros((1, 1, 1500))
        sentence = ''
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        for i in range(20):
            output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
            states_value = [h, c]
            output_tokens = output_tokens.reshape(num_decoder_tokens)
            y_hat = np.argmax(output_tokens)
            if y_hat == 0:
                continue
            if inv_map[y_hat] is None:
                break
            else:
                sentence = sentence + inv_map[y_hat] + ' '
                target_seq = np.zeros((1, 1, 1500))
                target_seq[0, 0, y_hat] = 1
        return ' '.join(sentence.split()[:-1])

    def decode_sequence2bs(self, input_seq):
        states_value = self.inf_encoder_model.predict(input_seq)
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        self.beam_search(target_seq, states_value, [], [], 0)
        return decode_seq

    def beam_search(self, target_seq, states_value, prob, path, lens):
        global  max_probability, decode_seq
        node = 2
        output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
        output_tokens = output_tokens.reshape(num_decoder_tokens)
        sampled_token_index = output_tokens.argsort()[-node:][::-1]
        states_value = [h, c]
        for i in range(node):
            if sampled_token_index[i] == 0:
                sampled_char = ''
            else:
                sampled_char = list(self.tokenizer.word_index.keys())[list(self.tokenizer.word_index.values()).index(sampled_token_index[i])]
            MAX_LEN = 20
            if sampled_char != 'eos' and lens <= MAX_LEN:
                p = output_tokens[sampled_token_index[i]]
                if sampled_char == '':
                    p = 1
                prob_new = list(prob)
                prob_new.append(p)
                path_new = list(path)
                path_new.append(sampled_char)

                target_seq = np.zeros((1, 1, num_decoder_tokens))
                target_seq[0, 0, sampled_token_index[i]] = 1.
                self.beam_search(target_seq, states_value, prob_new, path_new, lens + 1)
            else:
                p = output_tokens[sampled_token_index[i]]
                prob_new = list(prob)
                prob_new.append(p)
                p = functools.reduce(operator.mul, prob_new, 1)
                if p > max_probability:
                    decode_seq = path
                    max_probability = p

    def decoded_sentence_tuning(self, decoded_sentence):
        decode_str = []
        filter_string = ['bos', 'eos']
        uni_gram = {}
        last_string = ""
        for idx2, c in enumerate(decoded_sentence):
            if c in uni_gram:
                uni_gram[c] += 1
            else:
                uni_gram[c] = 1
            if last_string == c and idx2 > 0:
                continue
            if c in filter_string:
                continue
            if len(c) > 0:
                decode_str.append(c)
            if idx2 > 0:
                last_string = c
        return decode_str

    def get_test_data(self):
        X_test = []
        X_test_filename = []
        with open(os.path.join(test_path, 'testing_id.txt')) as testing_file:
            lines = testing_file.readlines()
            for filename in lines:
                filename = filename.strip()
                f = np.load(os.path.join('/content/data/testing_data/video', 'feat', filename + '.npy'))
                X_test.append(f)
                X_test_filename.append(filename[:-4])
            X_test = np.array(X_test)
        return X_test, X_test_filename

    def test(self):
        global max_probability
        X_test, X_test_filename = self.get_test_data()
        for search_type in ['greedy', 'beam']:
            with open(os.path.join(test_path, f'test2_{search_type}.txt'), 'w') as file:
                for idx, x in enumerate(X_test):
                    file.write(X_test_filename[idx] + ',')
                    start = time.time()
                    if search_type == 'greedy':
                        decoded_sentence = self.greedy_search(x.reshape(-1, 80, 4096))
                    else:
                        decoded_sentence = self.decode_sequence2bs(x.reshape(-1, 80, 4096))
                        decode_str = self.decoded_sentence_tuning(decoded_sentence)
                        decoded_sentence = ' '.join(decode_str)
                    file.write(decoded_sentence + ',{:.2f}'.format(time.time() - start))
                    file.write('\n')

                    max_probability = -1

if __name__ == "__main__":
    video_to_text = VideoDescriber()
    video_to_text.test()


1/1 [==============================] - 0s 32ms/step


تقييم النموذج باستخدام مقاييس التقييم مثل
BLEU أو METEOR،
  هذا الكود يقوم بحساب درجة
  BLEU بين التعليقات التوضيحية المولدة والتعليقات التوضيحية الحقيقية:

الكود يقوم بحساب درجة BLEU بين الجملة المرجعية والجملة المولدة. الدرجة تتراوح بين 0 و1 حيث 1 يعني أن الجملتين متطابقتين تمامًا.

الخطوات هي كالتالي:

يتم استيراد وظيفة sentence_bleu من مكتبة nltk.translate.bleu_score.

يتم تقسيم الجمل المرجعية والمولدة إلى قائمة من الكلمات.

يتم حساب درجة BLEU باستخدام الجمل المرجعية والجملة المولدة.
يتم طباعة الدرجة.

بالنسبة للنتيجة، القيمة 7.380245217279165e-78 هي قيمة صغيرة جدًا بالقرب من الصفر

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu_score(reference_caption, generated_caption):
    """
    Calculate BLEU score between the reference caption and the generated caption.
    """
    reference = [reference_caption.split()]
    candidate = generated_caption.split()
    score = sentence_bleu(reference, candidate)
    return score

reference_caption = "this is a test caption"
generated_caption = "this is a generated caption"
score = calculate_bleu_score(reference_caption, generated_caption)
print(f"Bleu Score: {score}")


Bleu Score: 7.380245217279165e-78


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

def calculate_rouge_score(reference_caption, generated_caption):
    """
    Calculate ROUGE score between the reference caption and the generated caption.
    """
    rouge = Rouge()
    scores = rouge.get_scores(generated_caption, reference_caption, avg=True)
    return scores

reference_caption = "this is a test caption"
generated_caption = "this is a generated caption"
scores = calculate_rouge_score(reference_caption, generated_caption)
print(f"Rouge Scores: {scores}")


Rouge Scores: {'rouge-1': {'r': 0.8, 'p': 0.8, 'f': 0.7999999950000002}, 'rouge-2': {'r': 0.5, 'p': 0.5, 'f': 0.4999999950000001}, 'rouge-l': {'r': 0.8, 'p': 0.8, 'f': 0.7999999950000002}}


In [ ]:
def extract_features1(video, model):
    # Convert the video to a sequence of images
    image_list = video_to_frames(video)

    # Initialize an array to hold the features
    features = np.zeros((len(image_list), 4096))

    # Process each image
    for i, image_path in enumerate(image_list):
        # Load and preprocess the image
        img = load_image(image_path)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)

        # Extract the features
        feature = model.predict(img)

        # Flatten the features and store in the array
        features[i] = feature.flatten()

    return features


In [ ]:
def extract_features2(video_path, model):
    video_id = video_path.split("/")[-1].split(".")[0]
    print(video_id)
    print(f'Processing video {video_path}')

    image_list = video_to_frames(video_path)
    samples = np.round(np.linspace(
        0, len(image_list) - 1, 80))
    image_list = [image_list[int(sample)] for sample in samples]
    images = np.zeros((len(image_list), 224, 224, 3))
    for i in range(len(image_list)):
        img = load_image(image_list[i])
        images[i] = img
    images = np.array(images)
    fc_feats = model.predict(images, batch_size=128)
    img_feats = np.array(fc_feats)
    shutil.rmtree(os.path.join(base_path, 'temporary_images'))
    return img_feats


In [ ]:
model = model_cnn_load()
video_features = extract_features2('/content/testing_data/video/j2Dhf-xFUxU_13_20.avi', model)

In [ ]:
from keras.models import load_model
from nltk.translate.bleu_score import sentence_bleu
import joblib
import os

# Load the models and tokenizer
encoder_model = load_model(os.path.join(save_model_path, 'encoder_model.keras'))
decoder_model = load_model(os.path.join(save_model_path, 'decoder_model.h5'))
tokenizer = joblib.load(os.path.join(save_model_path, 'tokenizer1500'))

# Function to generate a caption for a video
def generate_caption(video, encoder_model, decoder_model, tokenizer):
    # Convert the video to a sequence of image features
    video_features = extract_features2(video, encoder_model)

    # Generate the caption using the decoder model
    generated_caption = decoder_model(video_features, decoder_model, tokenizer)

    return generated_caption

# Calculate the BLEU score
reference_caption = "A women is cutting bread"
generated_caption = generate_caption('/content/testing_data/video/j2Dhf-xFUxU_13_20.avi', encoder_model, decoder_model, tokenizer)
score = sentence_bleu([reference_caption.split()], generated_caption.split())
print(f"Bleu Score: {score}")


NameError: name 'preprocess_input' is not defined

In [ ]:
#!zip -r /content/model_final.zip /content/model_final/

  adding: content/model_final/ (stored 0%)
  adding: content/model_final/decoder_model.keras (deflated 7%)
  adding: content/model_final/tokenizer1500 (deflated 52%)
  adding: content/model_final/model.h5 (deflated 8%)
  adding: content/model_final/decoder_model.h5 (deflated 7%)
  adding: content/model_final/encoder_model.h5 (deflated 8%)
  adding: content/model_final/decoder_model_weights.h5 (deflated 7%)
  adding: content/model_final/encoder_model.keras (deflated 8%)


In [ ]:
!zip -r /content/YouTubeClips.zip /content/YouTubeClips/

  adding: content/YouTubeClips/ (stored 0%)
  adding: content/YouTubeClips/PeUHy0A1GF0_51_55.avi (deflated 3%)
  adding: content/YouTubeClips/Xvy21KrDv8I_0_10.avi (deflated 3%)
  adding: content/YouTubeClips/sb1dX5xAFvk_10_17.avi (deflated 0%)
  adding: content/YouTubeClips/uGDuIyfJXXg_23_33.avi (deflated 4%)
  adding: content/YouTubeClips/cCmnN96zIeQ_14_24.avi (deflated 2%)
  adding: content/YouTubeClips/ao-9B8IV9_E_87_95.avi (deflated 1%)
  adding: content/YouTubeClips/3SKyc0aKx70_46_52.avi (deflated 1%)
  adding: content/YouTubeClips/Je3V7U5Ctj4_488_495.avi (deflated 0%)
  adding: content/YouTubeClips/R2DvpPTfl-E_20_25.avi (deflated 3%)
  adding: content/YouTubeClips/WqQonRVs7WA_0_10.avi (deflated 1%)
  adding: content/YouTubeClips/omIPdpxg--4_39_46.avi (deflated 3%)
  adding: content/YouTubeClips/EwTZ2xpQwpA_14_21.avi (deflated 3%)
  adding: content/YouTubeClips/u4T76jsPin0_0_11.avi (deflated 1%)
  adding: content/YouTubeClips/t4vP-cXXWkY_14_20.avi (deflated 1%)
  adding: content/Y

. هذا الكود يستخدم نموذج VGG16 المدرب مسبقًا ويقوم بتجميد الطبقات حتى الطبقة الأخيرة:

In [ ]:
from keras.applications import VGG16

# Load the VGG16 network, ensuring the head FC layer sets are left off
baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the base model
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(num_classes, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will not be updated during the training process
for layer in baseModel.layers:
    layer.trainable = False
